# Install requirements

In [ ]:
!pip install tensorflow_addons numpy pandas tensorflow sklearn nltk spacy textblob gensim scipy seaborn matplotlib minio mlflow wordcloud boto3


In [ ]:
import os
cwd = os.getcwd()

cwd

In [1]:
import sys
# sys.path is a list of absolute path strings
sys.path.append('/opt/app-root/src/anz_ml_project/')
from src.loadingdata.read_dataset import ReadData
from src.features.build_features import BuildFeatures
from src.modules.build_model import BuildModel
from src.modules.train_model import MLflow, TrainModel
from src.modules.predict_model import Predictor,Transformer,DownloadArtifact

# from src.modules.predict_model import BuildModel
# from src.modules.train_model import BuildModel


# Load Libraries

In [2]:
# import sys
# # sys.path is a list of absolute path strings
# sys.path.append('/opt/app-root/src/anz_ml_project/')

# from src.features.build_features import BuildFeatures

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn.feature_extraction.text as text
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from textblob import TextBlob
from nltk.stem import PorterStemmer,SnowballStemmer
from textblob import Word
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
# from wordcloud import WordCloudfrom wordcloud import WordCloud
from io import StringIO
import string
import gensim
from gensim.models import Word2Vec
import itertools
import scipy
from scipy import spatial
import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
import joblib

import mlflow
import warnings

from minio import Minio
import subprocess
import ipynbname
warnings.filterwarnings("ignore")
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)


tokenizer = ToktokTokenizer()
# stopword_list = nltk.download('stopwords')

# Define a class to read the dataset

# Define a class to preprocess the data and make them ready for modeling

# Define a class for building the Deep learning based model for  NLP 

# Define a class to configur MLFLOW

# Define a class for training the model and tracking it with MLflow

# Define classes for Deploy simulations

# download  artifacts for testing

# Initialize the config file for mlflow and Minio

In [3]:

HOST = "http://mlflow:5500"

PROJECT_NAME = "NlpTc"
EXPERIMENT_NAME = "NlpLstm"

os.environ['MLFLOW_S3_ENDPOINT_URL']='http://minio-ml-workshop:9000'
os.environ['AWS_ACCESS_KEY_ID']='minio'
os.environ['AWS_SECRET_ACCESS_KEY']='minio123'
os.environ['AWS_REGION']='us-east-1'
os.environ['AWS_BUCKET_NAME']='raw-data-saeed'



## Define a Function to read from Minio S3 Bucket

In [4]:
def get_s3_server():
    minioClient = Minio('minio-ml-workshop:9000',
                    access_key='minio',
                    secret_key='minio123',
                    secure=False)

    return minioClient

In [5]:
client = get_s3_server()

## SetUp MLFlow to track the model

In [6]:
mlflow = MLflow(mlflow, HOST,EXPERIMENT_NAME).SetUp_Mlflow()

# Readinng the data


In [7]:
train_data, test_data, train_labels, test_labels,enc = ReadData(S3BucketName = "raw-data-saeed",FILE_NAME="data.csv").ReadDataFrameData()

['Bank account or service' 'Consumer Loan' 'Credit card'
 'Credit reporting' 'Debt collection' 'Money transfers' 'Mortgage'
 'Other financial service' 'Payday loan' 'Prepaid card' 'Student loan']
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]), array([119,  90, 184, 242, 436,  22, 400,   2,  26,  10,  69]))
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]), array([ 30,  22,  46,  61, 109,   5, 100,   1,   7,   2,  17]))


In [ ]:
joblib.dump(enc, 'labelencoder.pkl')  

In [8]:
# from src.features.build_features import BuildFeatures
train_data.shape

test_labels.shape


(400,)

# Prepare data for modeling

In [9]:
BFCLASS = BuildFeatures(TRAIN_DATA=train_data,TEST_DATA=test_data,TRAIN_LABELS=train_labels,TEST_LABELS=test_labels, GloveData="glove.6B.50d.txt",EMBEDDING_DIM=50, WEIGHT_FLAG = False,MLFLOW_S3_ENDPOINT_URL = "minio-ml-workshop:9000",AWS_ACCESS_KEY_ID='minio',AWS_SECRET_ACCESS_KEY = 'minio123',SECURE = False)

In [10]:
train_data,test_data,train_labels , test_labels,word_index,tokenizer,MAX_SEQUENCE_LENGTH = BFCLASS.PreProcessingTextData()

Maximum Sequence length is 348 .
Found 7453 unique tokens.
Shape of train data tensor: (1600, 348)
Shape of train label tensor: (1600, 11)
Shape of test label tensor: (400, 11)


In [ ]:
joblib.dump(tokenizer, 'tokenizer.pkl')  

# Deep Learning define, train and test model


## Define the Model

In [11]:
model = BuildModel(WORD_INDEX=word_index, MAX_SEQUENCE_LENGTH=MAX_SEQUENCE_LENGTH, EMWEIGHTS=[]).SetupModel()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 348, 50)           372700    
_________________________________________________________________
bidirectional (Bidirectional (None, 348, 200)          120800    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               935936    
_________________________________________________________________
dense (Dense)                (None, 11)                5643      
Total params: 1,435,079
Trainable params: 1,435,079
Non-trainable params: 0
_________________________________________________________________


## Train the Model

In [13]:
model, history= TrainModel(model, tokenizer, enc,train_data, train_labels,test_data, test_labels,HOST, EXPERIMENT_NAME, BATCH_SIZE=64,EPOCHS=1).ModelTraining()

25/25 [==============================] - 37s 1s/step - loss: 1.9930 - acc: 0.3250 - val_loss: 1.6550 - val_acc: 0.4850
INFO:tensorflow:Assets written to: /tmp/tmpkqbdhntl/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmpkqbdhntl/model/data/model/assets
Traceback (most recent call last):
  File "/opt/app-root/lib64/python3.9/site-packages/urllib3/connectionpool.py", line 465, in _make_request
    assert_header_parsing(httplib_response.msg)
  File "/opt/app-root/lib64/python3.9/site-packages/urllib3/util/response.py", line 91, in assert_header_parsing
    raise HeaderParsingError(defects=defects, unparsed_data=unparsed_data)
urllib3.exceptions.HeaderParsingError: [MissingHeaderBodySeparatorDefect()], unparsed data: 'HTTP/1.1 200 OK\r\nAccept-Ranges: bytes\r\nContent-Length: 0\r\nContent-Security-Policy: block-all-mixed-content\r\nETag: "61d1d8eae962c9cee3ae5ce925da8cd4"\r\nServer: MinIO\r\nStrict-Transport-Security: max-age=31536000; includeSubDomains\r\nVary: Origin\r\nVary: Accept-Encoding\r\nX-Amz-Request-Id: 16B263BD971D1766\r\nX-Content-Type-Options: nosniff\r\nX-Xss-Protection: 1; mode=block\r\nDate: Fri, 29 Oct 2021 03:48:00 GMT\r\n\r\n'


FileNotFoundError: [Errno 2] No such file or directory: 'tokenizer.pkl'

### Plot the training and testing Loss

In [ ]:
fig1 = plt.figure()
plt.plot(history.history['loss'],'r',linewidth=3.0)
plt.plot(history.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves :RNN - LSTM',fontsize=16)
plt.show()

#### Plot the training and testing Accuracy

In [ ]:
fig1 = plt.figure()
plt.plot(history.history['acc'],'r',linewidth=3.0)
plt.plot(history.history['val_acc'],'b',linewidth=3.0)
plt.legend(['Training acc', 'Validation acc'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves :RNN - LSTM',fontsize=16)
plt.show()

## Test the model

In [ ]:
#predictions on test data
predicted=model.predict(test_data)
predicted

In [ ]:
#model evaluation
import sklearn
from sklearn.metrics import precision_recall_fscore_support as score
precision, recall, fscore, support = score(test_labels, predicted.round())
print('precision: \n{}'.format(precision))
print('recall: \n{}'.format(recall))
print('fscore: \n{}'.format(fscore))
print('support: \n{}'.format(support))
print("############################")

In [ ]:
from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(test_labels.argmax(axis=1), predicted.argmax(axis=1))
fig, ax = plt.subplots(figsize=(8,6))
sns.heatmap(conf_mat, annot=True, fmt="d", cmap="BuPu",xticklabels=enc.classes_,yticklabels=enc.classes_)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

### Download the artifacts

In [ ]:

DownloadArtifact(mlflow, MODEL_NAME='lstmt1', MODEL_VRSION='1').download_artifacts()



###  Test with Actual  data

#### Define a sample

In [ ]:
sample_data = {"data":
  {


        "names":
            [
              "Debt collection"
            ],
      "ndarray": ["could longer pay enormous charge hired company nl take either nothing pay day loan company accept term get several letter week threatened take civil action get check"]

  }
}


#### Transform the data


In [ ]:
ready_data = Transformer().transform_input(sample_data,"name","meta")

### Test the prediction

In [ ]:
output = Predictor().predict(ready_data,"features")

In [ ]:
output = Predictor().predict(ready_data,ready_data)

In [ ]:
model.predict(ready_data)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_labels, predicted.round(),target_names=enc.classes_))


After hours of training we get good results with LSTM(type of recurrent neural network) compared to CNN. From the learning curves it is clear the model needs to be tuned for overfitting by selecting hyperparameters such as no of epochs via early stopping and dropout for regularization.

We could further improve our final result by ensembling our xgboost and Neural network models by using Logistic Regression as our base model.

